In [1]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_FilmTrust'

In [2]:
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/filmtrust'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pickle
from sklearn import metrics

In [4]:
ratings=pd.read_csv(f'{DATAPATH}/ratings.txt',sep=' ',names=['userId','itemId','rating'])
print (ratings.head())

   userId  itemId  rating
0       1       1     2.0
1       1       2     4.0
2       1       3     3.5
3       1       4     3.0
4       1       5     4.0


In [5]:
ratings.shape

(35497, 3)

In [6]:
ratings.drop_duplicates(inplace=True)

In [7]:
ratings[['userId','itemId']].duplicated().value_counts()

False    35494
True         2
dtype: int64

In [8]:
ratings=ratings.groupby(['userId','itemId'],as_index=False)['rating'].mean()

In [9]:
ratings.shape

(35494, 3)

In [10]:
trust=pd.read_csv(f'{DATAPATH}/trust.txt',sep=' ',names=['userId','userId_1','trust'])
print (trust.head())

   userId  userId_1  trust
0       2       966      1
1       2       104      1
2       5      1509      1
3       6      1192      1
4       7      1510      1


In [11]:
ratings.shape, trust.shape

((35494, 3), (1853, 3))

In [12]:
trust_agg=trust.groupby('userId')['trust'].agg(['mean','min','max','sum'])

In [13]:
trust_agg.columns+='_on_other'

In [14]:
trust_agg.head()

,mean_on_other,min_on_other,max_on_other,sum_on_other
userId,,,,
2,1,1,1,2
5,1,1,1,1
6,1,1,1,1
7,1,1,1,1
12,1,1,1,1


In [15]:
trust_agg_on_self=trust.groupby('userId_1')['trust'].agg(['mean','min','max','sum'])

In [16]:
trust_agg_on_self.columns+='_on_self'

In [17]:
trust_agg_on_self.head()

,mean_on_self,min_on_self,max_on_self,sum_on_self
userId_1,,,,
2,1,1,1,1
4,1,1,1,1
6,1,1,1,5
13,1,1,1,1
16,1,1,1,1


In [18]:
trust_agg_on_self.reset_index(inplace=True)

In [19]:
trust_agg.reset_index(inplace=True)

In [20]:
trust_agg_on_self.rename({'userId_1':'userId'},axis=1,inplace=True)

In [21]:
trust_agg_on_self.head()

,userId,mean_on_self,min_on_self,max_on_self,sum_on_self
0,2,1,1,1,1
1,4,1,1,1,1
2,6,1,1,1,5
3,13,1,1,1,1
4,16,1,1,1,1


In [22]:
ratings.shape

(35494, 3)

In [23]:
trust_agg_on_self.columns

Index(['userId', 'mean_on_self', 'min_on_self', 'max_on_self', 'sum_on_self'], dtype='object')

In [24]:
trust_agg.columns

Index(['userId', 'mean_on_other', 'min_on_other', 'max_on_other',
       'sum_on_other'],
      dtype='object')

In [25]:
trust_agg.columns='f_'+trust_agg.columns

In [26]:
trust_agg.rename({'f_userId':'userId'},axis=1,inplace=True)

In [27]:
trust_agg.columns

Index(['userId', 'f_mean_on_other', 'f_min_on_other', 'f_max_on_other',
       'f_sum_on_other'],
      dtype='object')

In [28]:
trust_agg_on_self.columns='f_'+trust_agg_on_self.columns

In [29]:
trust_agg_on_self.rename({'f_userId':'userId'},axis=1,inplace=True)

In [30]:
trust_agg_on_self.columns

Index(['userId', 'f_mean_on_self', 'f_min_on_self', 'f_max_on_self',
       'f_sum_on_self'],
      dtype='object')

In [31]:
print(ratings.shape)
ratings=pd.merge(ratings,trust_agg_on_self,on='userId',how='left')
print(ratings.shape)

(35494, 3)
(35494, 7)


In [32]:
print(ratings.shape)
ratings=pd.merge(ratings,trust_agg,on='userId',how='left')
print(ratings.shape)

(35494, 7)
(35494, 11)


In [33]:
ratings.tail()

,userId,itemId,rating,f_mean_on_self,f_min_on_self,f_max_on_self,f_sum_on_self,f_mean_on_other,f_min_on_other,f_max_on_other,f_sum_on_other
35489,1508,669,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0
35490,1508,686,2.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0
35491,1508,693,3.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0
35492,1508,751,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0
35493,1508,806,3.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0


In [34]:
ratings.head()

,userId,itemId,rating,f_mean_on_self,f_min_on_self,f_max_on_self,f_sum_on_self,f_mean_on_other,f_min_on_other,f_max_on_other,f_sum_on_other
0,1,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
ratings.shape

(35494, 11)

In [36]:
dfratings=ratings.copy()

In [37]:
ratings.head()

,userId,itemId,rating,f_mean_on_self,f_min_on_self,f_max_on_self,f_sum_on_self,f_mean_on_other,f_min_on_other,f_max_on_other,f_sum_on_other
0,1,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### CROSS-VALIDATION - SIMULATE COLD START

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
ratings['rating'].value_counts()

4.00    9169
3.00    7876
3.50    7141
2.50    4392
2.00    3113
1.50    1600
1.00    1141
0.50    1060
3.25       1
2.75       1
Name: rating, dtype: int64

In [40]:
ratings.head()

,userId,itemId,rating,f_mean_on_self,f_min_on_self,f_max_on_self,f_sum_on_self,f_mean_on_other,f_min_on_other,f_max_on_other,f_sum_on_other
0,1,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
dfratings=ratings.copy()

In [42]:
users=dfratings['userId'].unique()

In [43]:
users=np.random.permutation(users)

In [44]:
normalfrac=0.9

In [45]:
users_normal=users[:np.int(np.round(len(users)*normalfrac))]

In [46]:
dfnormal=dfratings.loc[dfratings['userId'].isin(users_normal)]

In [47]:
dfsimulate=dfratings.loc[~dfratings['userId'].isin(users_normal)]

In [48]:
dfnormal.shape, dfsimulate.shape

((31779, 11), (3715, 11))

In [49]:
temptrain,tempvalid=train_test_split(dfnormal,test_size=0.1,train_size=0.9,random_state=11,shuffle=True)

In [50]:
temptrain['random_dstype']='train'
tempvalid['random_dstype']='test'

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [51]:
dfnormal=pd.concat([temptrain,tempvalid],axis=0)

In [52]:
dfnormal.shape

(31779, 12)

In [53]:
dfnormal['random_dstype'].value_counts()

train    28601
test      3178
Name: random_dstype, dtype: int64

In [54]:
dfsimulate.shape

(3715, 11)

In [56]:
simulateusers=dfsimulate['userId'].unique()

In [58]:
simulateusers=np.random.permutation(simulateusers)

In [60]:
simulateusers_train=simulateusers[:np.int(np.round(len(simulateusers)*normalfrac))]

In [61]:
dfsimulate_train=dfsimulate.loc[dfsimulate['userId'].isin(simulateusers_train)]
dfsimulate_test=dfsimulate.loc[~dfsimulate['userId'].isin(simulateusers_train)]

In [64]:
dfsimulate_train['random_dstype']='train'
dfsimulate_test['random_dstype']='test'

In [65]:
dfsimulate=pd.concat([dfsimulate_train,dfsimulate_test],axis=0)

In [66]:
dfratings=pd.concat([dfnormal,dfsimulate],axis=0)

In [67]:
dftrain=dfratings.loc[dfratings['random_dstype']=='train']

In [68]:
dfvalid=dfratings.loc[dfratings['random_dstype']!='train']

In [69]:
dftrain.shape, dfvalid.shape

((32036, 12), (3458, 12))

In [70]:
dfvalid['random_dstype'].value_counts()

test    3458
Name: random_dstype, dtype: int64

In [71]:
dftrain['flag_train']=1
dfvalid['flag_train']=0

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Missing value treatment

In [72]:
user_features=dftrain.columns[dftrain.columns.str.startswith('f_')]

In [73]:
user_features

Index(['f_mean_on_self', 'f_min_on_self', 'f_max_on_self', 'f_sum_on_self',
       'f_mean_on_other', 'f_min_on_other', 'f_max_on_other',
       'f_sum_on_other'],
      dtype='object')

In [74]:
dftrain[user_features].describe().T

,count,mean,std,min,25%,50%,75%,max
f_mean_on_self,14372.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
f_min_on_self,14372.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
f_max_on_self,14372.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
f_sum_on_self,14372.0,3.899109,5.564869,1.0,1.0,2.0,4.0,46.0
f_mean_on_other,13102.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
f_min_on_other,13102.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
f_max_on_other,13102.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
f_sum_on_other,13102.0,4.525950,5.592425,1.0,1.0,2.0,5.0,36.0


In [75]:
for myNm in user_features:
    dftrain['f_na_'+myNm[2:]]=dftrain[myNm].isnull()
    dftrain[myNm].fillna(0,inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/kirana/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [76]:
dftrain[user_features].describe().T

,count,mean,std,min,25%,50%,75%,max
f_mean_on_self,32036.0,0.448620,0.497361,0.0,0.0,0.0,1.0,1.0
f_min_on_self,32036.0,0.448620,0.497361,0.0,0.0,0.0,1.0,1.0
f_max_on_self,32036.0,0.448620,0.497361,0.0,0.0,0.0,1.0,1.0
f_sum_on_self,32036.0,1.749220,4.201545,0.0,0.0,0.0,1.0,46.0
f_mean_on_other,32036.0,0.408977,0.491653,0.0,0.0,0.0,1.0,1.0
f_min_on_other,32036.0,0.408977,0.491653,0.0,0.0,0.0,1.0,1.0
f_max_on_other,32036.0,0.408977,0.491653,0.0,0.0,0.0,1.0,1.0
f_sum_on_other,32036.0,1.851011,4.212099,0.0,0.0,0.0,2.0,36.0


In [77]:
for myNm in user_features:
    dfvalid['f_na_'+myNm[2:]]=dfvalid[myNm].isnull()
    dfvalid[myNm].fillna(0,inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [78]:
pickle.dump([dftrain,dfvalid],open(f'{DATAPATH}/df.pkl','wb'))